<a href="https://colab.research.google.com/github/Jordy-20035/hw_1/blob/main/%D0%9A%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%BE%D0%B2_%D0%BF%D0%BE_%D1%82%D0%BE%D0%BF%D0%B8%D0%BA%D0%B0%D0%BC_%D1%81_%D0%B8%D1%81%D0%BF%D0%BE%D0%BB%D1%8C%D0%B7%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%D0%BC_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip uninstall pymorphy2 -y

Found existing installation: pymorphy2 0.9.1
Uninstalling pymorphy2-0.9.1:
  Successfully uninstalled pymorphy2-0.9.1


In [18]:
!pip install corus py imbalanced-learn pymorphy3 pymorphy3-dicts-ru joblib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 52.2 MB/s eta 0:00:00


In [1]:
import random
import numpy as np
import pandas as pd
import corus
import re
import nltk
import string
import pymorphy3
from pymorphy3 import MorphAnalyzer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from imblearn.over_sampling import SMOTE

Fixed Random Seeds: Ensures consistent data splits and model initialization

Justification: Required for reproducible ML experiments

In [2]:
# Fix random seed for reproducibility
random_state = 42
np.random.seed(random_state)
random.seed(random_state)
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Initialize pymorphy3 lemmatizer
morph = pymorphy3.MorphAnalyzer()

In [4]:
# Function to lemmatize text using pymorphy3
def lemmatize_text(text):
    return " ".join([morph.parse(word)[0].normal_form for word in text.split()])

In [5]:
# Download the Lenta dataset
!wget -q https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = list(load_lenta(path))

In [6]:
# Load dataset into DataFrame
def load_lenta_data(limit=200000):
    data = [(record.title, record.text, record.topic) for record in records[:limit]]
    return pd.DataFrame(data, columns=["title", "text", "topic"])

df = load_lenta_data()

In [7]:
# Function for basic text preprocessing
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r"\d+", "", text)  # Remove digits
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    return lemmatize_text(text)  # Lemmatize text

In [8]:
# Remove missing and empty values
df = df.dropna()
df = df[df["text"].str.strip() != ""]

In [ ]:
# Apply text preprocessing
df["processed_text"] = df["text"].apply(preprocess_text)

In [ ]:
# Encode categorical topics
df["processed_topic"] = df["topic"].astype("category").cat.codes

In [ ]:
# Filter out underrepresented categories
category_counts = df["processed_topic"].value_counts()
valid_categories = category_counts[category_counts >= 2].index
df = df[df["processed_topic"].isin(valid_categories)]

In [ ]:
# Split dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(
    df["processed_text"], df["processed_topic"], test_size=0.4, stratify=df["processed_topic"], random_state=random_state
)

In [ ]:
# Apply SMOTE for balancing classes
smote = SMOTE(random_state=random_state)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train.to_frame(), y_train)
X_train_resampled = X_train_resampled.squeeze()

In [ ]:
# Further split validation and test sets
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=random_state
)

In [ ]:
# Dummy classifier for baseline comparison
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train_resampled, y_train_resampled)
y_pred_dummy = dummy.predict(X_test)
print("Dummy accuracy:", accuracy_score(y_test, y_pred_dummy))

In [ ]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2), sublinear_tf=True)

In [ ]:
# Define classification models
models = {
    "Logistic Regression": LogisticRegression(C=1.0, solver="liblinear", random_state=random_state),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=random_state),
    "Naïve Bayes": MultinomialNB()
}

In [ ]:
# Evaluate models using cross-validation
for model_name, model in models.items():
    pipeline = Pipeline([
        ("vectorizer", vectorizer),
        ("classifier", model)
    ])
    scores = cross_val_score(pipeline, X_train_resampled, y_train_resampled, cv=5, scoring="accuracy")
    print(f"{model_name} mean accuracy: {np.mean(scores):.4f}")

In [ ]:
# Train final model on best-performing classifier
final_pipeline = Pipeline([
    ("vectorizer", vectorizer),
    ("classifier", LogisticRegression(C=1.0, solver="liblinear", random_state=random_state))
])
final_pipeline.fit(X_train_resampled, y_train_resampled)
y_pred = final_pipeline.predict(X_test)
print("Final accuracy:", accuracy_score(y_test, y_pred))